In [1]:
import os

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="6"

In [2]:
import pandas as pd
import numpy as np

from tensorflow.keras.utils import to_categorical 
from sklearn.model_selection import train_test_split 
from tensorflow.keras.layers import Layer, Input,Dense,Flatten
from tensorflow.keras.models import Model 

import tensorflow as tf

In [5]:
INPUT_SIZE = 28

def create_model():
    input_tensor=Input(shape=(INPUT_SIZE,INPUT_SIZE))

    x = Flatten()(input_tensor)
    x = Dense(100,activation='relu')(x)
    x = Dense(30,activation='relu')(x)
    output = Dense(10,activation='softmax')(x)

    model = Model(inputs=input_tensor,outputs=output)

    return model

model = create_model()
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 784)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 100)               78500     
_________________________________________________________________
dense_3 (Dense)              (None, 30)                3030      
_________________________________________________________________
dense_4 (Dense)              (None, 10)                310       
Total params: 81,840
Trainable params: 81,840
Non-trainable params: 0
_________________________________________________________________


In [6]:
#0 ~ 1 사이값의 float32로 변경

def get_preprocessed_data(images,labels):
    images = np.array(images/255.0,dtype=np.float32)
    labels = np.array(labels,dtype=np.float32)
    return images,labels


# 0 ~ 사이값 float32로 변경하는 함수를 호출한 뒤 ohe 적용

def get_preprocessed_ohe(images,labels):
    images,labels=get_preprocessed_data(images,labels)
    oh_labels=to_categorical(labels)
    return images,labels


# 학습, 검증, 테스트 데이터셋에 전처리 및 OHE 적용

def get_train_valid_test_set(train_images,train_labels,test_images,test_labels,valid_size=0.15,random_state=2021):
    train_images,train_oh_labels=get_preprocessed_ohe(train_images,train_labels)
    test_images,test_oh_labels = get_preprocessed_ohe(test_images,test_labels)

    #검증 데이터에 데해
    tr_images,val_images,tr_oh_labels,val_oh_labels = train_test_split(train_images,train_oh_labels,test_size=valid_size,random_state=2021)
    return (tr_images,tr_oh_labels),(val_images,val_oh_labels),(test_images,test_oh_labels)

In [14]:
from tensorflow.keras.datasets import fashion_mnist 

fmist = fashion_mnist.load_data()

(train_images,train_labels),(test_images,test_labels) = fashion_mnist.load_data()

In [17]:
(tr_images,tr_oh_labels),(val_images,val_oh_labels),(test_images,test_oh_labels)= get_train_valid_test_set(train_images,train_labels,test_images,test_labels,valid_size=0.15,random_state=2021)

#### Callback 개요
- 사용자는 특정 Callback API를 활용하여 model.fit(callbacks=[...])로 등록 시키면
  fit() iteration 시 특정 이벤트 발생 시마다 등록된 callback이 호출되어 수행

  주로 최적의 Learning rate를 Control하는 데 사용


- ModelCheckpoint()
- ReduceLROnPlateua()
- LearningRateScheduler()
- EarlyStopping()
- TensorBoard()

#### 대표적인 Callback : ModelCheckpoint

ModelCheckpoint(filepath,monitor='val_loss','verbose=0, save_best_only=False,save_weights_only =False, mode=;auto, period=1)



- 특정 조건에 맞춰서 모델을 파일로 저장
- filepath = filepath는 (on_epoch_end에서 전달되는) epoch의 값과 logs의 키로 채워진 이름 형식 옵션을 가질 수 있다. 예를 들어 filepath가 weights.{epoch:02d}-{val_loss:.2f}.hdf5라면, 파일 이름에 세대 번호와 검증 손실을 넣어 모델의 체크포인트가 저장

- monitor : 모니터할 지표(loss 혹은 평가지표)
- save_best_only : 가장 좋은 성능을 나타넬 모델만 저장할 지 False로 해두는게 좋음
- save_weights_only : Weights만 저장할 지 True로 해두는게 좋고
- mode: {auto,min,max} 중 하나. monitor 지표가 감소해야 좋을 경우 min, 증가해야 좋을 경우 amx, auto는 monitor 이름에서 자동으로 유추

In [10]:
!pwd

/home/key2317/main/Deep_Learning/PracticeFreely


In [19]:
from tensorflow.keras.callbacks import ModelCheckpoint 
from tensorflow.keras.optimizers import Adam

model = create_model()

model.compile(optimizer=Adam(0.001),loss='categorical_crossentropy',metrics=['Accuracy'])

mcp_cb = ModelCheckpoint(filepath='/home/key2317/main/Deep_Learning/PracticeFreely/weights.{epoch:02d}-{val_loss:.2f}.hdf5',monitor='val_loss', 
save_best_only=True, save_weights_only=True, mode='min',period=1,verbose=1)

hisotry = model.fit(x=tr_images,y=tr_oh_labels,batch_size=128,epochs=10,validation_data=(val_images,val_oh_labels),
callbacks=[mcp_cb])


Epoch 1/10


ValueError: in user code:

    /home/key2317/anaconda3/lib/python3.8/site-packages/keras/engine/training.py:853 train_function  *
        return step_function(self, iterator)
    /home/key2317/anaconda3/lib/python3.8/site-packages/keras/engine/training.py:842 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    /home/key2317/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:1286 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    /home/key2317/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:2849 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    /home/key2317/anaconda3/lib/python3.8/site-packages/tensorflow/python/distribute/distribute_lib.py:3632 _call_for_each_replica
        return fn(*args, **kwargs)
    /home/key2317/anaconda3/lib/python3.8/site-packages/keras/engine/training.py:835 run_step  **
 

#### ReduceLRPlateau
